In [92]:
import pandas as pd


data= pd.read_csv('/content/drive/MyDrive/diabetes.csv')
df = data.copy(deep = True)
df
df1 = df.select_dtypes(exclude=[object])
correlation = df1.corr()
round(correlation,3)
cor_target = abs(correlation["Outcome"])
relevent_features = cor_target[cor_target >= 0.20 ]
df = df[relevent_features.index]
print(df.shape)
df.columns

(768, 5)


Index(['Pregnancies', 'Glucose', 'BMI', 'Age', 'Outcome'], dtype='object')

In [93]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 1 )

In [ ]:
import keras as kr
model = kr.models.Sequential()
model.add(kr.layers.Dense(128,input_dim=4,activation="relu"))
model.add(kr.layers.Dense(64,activation="relu"))
model.add(kr.layers.Dense(32,activation="relu"))
model.add(kr.layers.Dense(1,activation="sigmoid"))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=100)

Epoch 1/100
39/39 [==============================] - 1s 2ms/step - loss: 1.5373 - accuracy: 0.5188
Epoch 2/100
39/39 [==============================] - 0s 2ms/step - loss: 0.6766 - accuracy: 0.6101
Epoch 3/100
39/39 [==============================] - 0s 2ms/step - loss: 0.6740 - accuracy: 0.6486
Epoch 4/100
39/39 [==============================] - 0s 2ms/step - loss: 0.7346 - accuracy: 0.5839
Epoch 5/100
39/39 [==============================] - 0s 2ms/step - loss: 0.6742 - accuracy: 0.6099
Epoch 6/100
39/39 [==============================] - 0s 2ms/step - loss: 0.7459 - accuracy: 0.6154
Epoch 7/100
39/39 [==============================] - 0s 2ms/step - loss: 0.6725 - accuracy: 0.6604
Epoch 8/100
39/39 [==============================] - 0s 2ms/step - loss: 0.6486 - accuracy: 0.6605
Epoch 9/100
39/39 [==============================] - 0s 2ms/step - loss: 0.6461 - accuracy: 0.6556
Epoch 10/100
39/39 [==============================] - 0s 2ms/step - loss: 0.6162 - accuracy: 0.6914
Epoch 11/

In [ ]:
y_pred = model.predict(x_test)
y_pred = y_pred.round()


In [ ]:
model.evaluate(x_test,y_test)

5/5 [==============================] - 0s 3ms/step - loss: 0.8667 - accuracy: 0.7922


[0.8667243719100952, 0.7922077775001526]

In [ ]:
model.save('annmodel')

INFO:tensorflow:Assets written to: annmodel/assets


In [94]:
model1 = kr.models.load_model('/content/annmodel')

In [95]:
model.evaluate(x_test,y_test)

5/5 [==============================] - 0s 3ms/step - loss: 0.8667 - accuracy: 0.7922


[0.8667243719100952, 0.7922077775001526]

In [96]:
x_test

,Pregnancies,Glucose,BMI,Age
285,7,136,26.0,51
101,1,151,26.1,22
581,6,109,25.0,27
352,3,61,34.4,46
726,1,116,36.1,25
...,...,...,...,...
563,6,99,26.9,32
318,3,115,38.1,28
154,8,188,47.9,43
684,5,136,0.0,69


In [98]:
model.predict([[7,136,26,51]])

array([[0.64852047]], dtype=float32)

In [1]:
import keras
import numpy as np
from flask import Flask,request,jsonify,render_template
app = Flask(__name__)
model = keras.models.load_model('annmodel', compile = False)
@app.route('/')
def home():
  return render_template('api.html')
@app.route('/predict',methods=['POST'])
def predict():
    int_features = [[float(x) for x in request.form.values()]]
    final_features = [[np.array(int_features)]]
    prediction = model.predict(int_features)

    return render_template('api.html',prediction_text='chance for diabetes : {} '.format(prediction))

if __name__ == '__main__':
    # Threaded option to enable multiple instances for multiple user access support
    app.run(threaded=True, port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Jun/2021 01:41:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2021 01:42:03] "POST /predict HTTP/1.1" 200 -
